In [1]:
%load_ext autoreload
%autoreload 2

In [31]:
import sys
sys.path.insert(0, "../")

import autogluon as ag
import mxnet as mx
import mxnet.gluon.nn as nn
import mxnet.ndarray as nd
import mxnet.gluon as gluon

### simulate image_classification.py opt

In [15]:
from util.arg_parser import get_parser
from image_classification import get_model
import image_classification
import re
import sys

In [16]:
args = '''--data-path \"/path/to/cifar/\"
--augmentation low
--batch-size 128
--clip-threshold 1.3
--dataset cifar10
--epochs 150
--gpus 0
--log-interval 50
--lr 0.02
--lr-mode cosine
--mode hybrid
--model meliusnet22
--optimizer adam
--warmup-epochs 5'''

In [17]:
arg_list = re.split(' |\n',args)

In [18]:
old_args = sys.argv[1:]
sys.argv[1:] = arg_list
parser = get_parser()
opt = parser.parse_args()
sys.argv[1:] = old_args

In [19]:
opt

Namespace(activation_method='det_sign', approximation='', augmentation='low', batch_size=128, bits=1, bits_a=1, block_config=None, builtin_profiler=0, clip_threshold=1.3, data_dir='', data_path='"/path/to/cifar/"', dataset='cifar10', dilated=False, downsample_structure=None, dry_run=False, dtype='float32', epochs=150, gpus='0', growth_rate=None, init_features=None, initial_layers='imagenet', initialization='gaussian', interrupt_at=None, kvstore='device', log='image-classification.log', log_interval=50, lr=0.02, lr_factor=0.1, lr_mode='cosine', lr_steps='30,60,90', mode='hybrid', model='meliusnet22', momentum=0.9, num_workers=4, optimizer='adam', plot_network=None, prefix='', profile=False, progress='', reduction=None, resume='', resume_states='', save_frequency=None, seed=123, start_epoch=0, test_run=False, use_pretrained=False, warmup_epochs=5, warmup_lr=0.0, wd=0.0, weight_quantization='det_sign', write_summary=None)

### Do actual stuff

In [26]:
from bmxnet_examples.datasets.imagenet import Imagenet
from bmxnet_examples.datasets.cifar10 import Cifar10

In [27]:
img_net = Cifar10()

In [39]:
train_data, val_data, batch_fn = img_net.get_data(opt)

In [28]:
type(train_data)

mxnet.gluon.data.dataloader.DataLoader

In [36]:
isinstance(train_data, gluon.data.dataloader.DataLoader)

True

In [41]:
for t in train_data:
    print(len(t))

2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2


In [44]:
from mxnet.gluon.nn import Conv2D

In [45]:
# Create 2 Conv2D layers with same arguments, so shape of parameters are same.
conv_layer_1 = Conv2D(kernel_size=(3, 3), padding=(1, 1), channels=32, activation="relu", bias_initializer='ones')
# we share weight, not bias
conv_layer_2 = Conv2D(kernel_size=(3, 3), padding=(1, 1), channels=32, activation="relu",
                      params=conv_layer_1.collect_params('conv0_weight'))

conv_layer_1.initialize()
conv_layer_2.initialize()

# unfortunately, names of the biases are also same, while they have different values.
# But in new versions MXNet loading works based on indices, not parameter names
print(conv_layer_1.collect_params())
print(conv_layer_2.collect_params())

# just showing that biases are different
print(conv_layer_1.bias.data())
print(conv_layer_2.bias.data())

conv0_ (
  Parameter conv0_weight (shape=(32, 0, 3, 3), dtype=<class 'numpy.float32'>)
  Parameter conv0_bias (shape=(32,), dtype=<class 'numpy.float32'>)
)
conv0_ (
  Parameter conv0_weight (shape=(32, 0, 3, 3), dtype=<class 'numpy.float32'>)
  Parameter conv0_bias (shape=(32,), dtype=<class 'numpy.float32'>)
)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1.]
<NDArray 32 @cpu(0)>

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]
<NDArray 32 @cpu(0)>


In [50]:
conv_layer_1.collect_params()

conv0_ (
  Parameter conv0_weight (shape=(32, 0, 3, 3), dtype=<class 'numpy.float32'>)
  Parameter conv0_bias (shape=(32,), dtype=<class 'numpy.float32'>)
)

In [49]:
conv_layer_1.collect_params('conv0_weight')

conv0_ (
  Parameter conv0_weight (shape=(32, 0, 3, 3), dtype=<class 'numpy.float32'>)
)